In [1]:
# ------------------------------ Packages & Files ------------------------------
from pathlib import Path

import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots

import ipywidgets as widgets
from IPython.display import display, HTML

from ppa_analysis import user_inputs, advanced_settings, hybrid, import_gen_data, import_emissions_data, \
     import_pricing_data

INFO: Using Python-MIP package version 1.15.0


In [2]:
# ------------------------------ Initialise the input collector ----------------
input_collector = user_inputs.launch_input_collector()

/home/nick/Documents/Github/ppa_analysis
data_caches/yearly_data_files


Dropdown(description='Year:', options=('2020', '2021'), value='2020')

Dropdown(description='Generator region:', options=('QLD1', 'NSW1', 'VIC1', 'SA1', 'TAS1'), value='QLD1')

Dropdown(description='Load region:', options=('QLD1', 'NSW1', 'VIC1', 'SA1', 'TAS1'), value='QLD1')

Dropdown(description='Load data file:', options=('(27) Foundry FN_same_year.csv', '(89) Water pumping CA.csv',…

...

BoundedFloatText(value=1.0, description='Matching percentile:')

FloatText(value=300.0, description='Exposure upper bound ($/MW/h):')

FloatText(value=20.0, description='Exposure upper bound ($/MW/h):')

Dropdown(description='Time series interval:', options=('60',), value='60')

Dropdown(description='Generator data set:', options=('GenCost 2018 Low',), value='GenCost 2018 Low')

In [3]:
generator_data_editor = user_inputs.launch_generator_data_editor(input_collector)

Output()

In [4]:
import pprint

pprint.pprint(generator_data_editor)

{'COOPGWF1: WIND - ONSHORE': {'label': <IPython.core.display.HTML object>},
 'CSPVPS1: PHOTOVOLTAIC FLAT PANEL': {'label': <IPython.core.display.HTML object>},
 'DDSF1: PHOTOVOLTAIC FLAT PANEL': {'label': <IPython.core.display.HTML object>},
 'KSP1: PHOTOVOLTAIC FLAT PANEL': {'label': <IPython.core.display.HTML object>},
 'out': Output(outputs=({'output_type': 'display_data', 'data': {'text/plain': '<IPython.core.display.HTML object>', 'text/html': '\n                    <h5>CSPVPS1: PHOTOVOLTAIC FLAT PANEL:</h5>\n                    '}, 'metadata': {}}, {'output_type': 'display_data', 'data': {'text/plain': '<IPython.core.display.HTML object>', 'text/html': '\n                    <h5>COOPGWF1: WIND - ONSHORE:</h5>\n                    '}, 'metadata': {}}, {'output_type': 'display_data', 'data': {'text/plain': '<IPython.core.display.HTML object>', 'text/html': '\n                    <h5>DDSF1: PHOTOVOLTAIC FLAT PANEL:</h5>\n                    '}, 'metadata': {}}, {'output_type': 'disp

In [5]:
# ----------------------------- Unpack user input ------------------------------
year_to_load_from_cache = input_collector['year'].value
GENERATOR_REGION = input_collector['generator_region'].value
LOAD_REGION = input_collector['load_region'].value
generators = list(input_collector['generators'].value)

# ----------------------------- Get Generation Data ----------------------------
gen_data_file = (
    advanced_settings.YEARLY_DATA_CACHE / 
    f'gen_data_{year_to_load_from_cache}.parquet'
)
gen_data = import_gen_data.get_preprocessed_gen_data(
    gen_data_file, [GENERATOR_REGION]
)
gen_data = gen_data[generators]

# ------------------------ Get Emissions Data ----------------------------------
emissions_data_file = (
    advanced_settings.YEARLY_DATA_CACHE / 
    f'emissions_data_{year_to_load_from_cache}.parquet'
)
emissions_intensity = import_emissions_data.get_preprocessed_avg_intensity_emissions_data(
    emissions_data_file, [LOAD_REGION, GENERATOR_REGION]
)

# ------------------------ Get Wholesale Price Data ----------------------------------
price_data_file = (
    advanced_settings.YEARLY_DATA_CACHE / 
    f'price_data_{year_to_load_from_cache}.parquet'
)
price_data = import_pricing_data.get_preprocessed_price_data(
    price_data_file, [LOAD_REGION, GENERATOR_REGION]
)

In [6]:
# -------- SIMPLE LCOE CALCULATIONS ----------

# Low assumption from GenCost 2018
lcoe_info_dict = {
    solar_1 : {
        'Nameplate Capacity (kW)' : 110000,
        'Fixed O&M ($/kW)': 14.4,
        'Variable O&M ($/kWh)' : 0.0,
        'Capital ($/kW)' : 1280,
        'Capacity Factor' : 0.22
    },
    solar_2 : {
        'Nameplate Capacity (kW)' : 125000,
        'Fixed O&M ($/kW)': 14.4,
        'Variable O&M ($/kWh)' : 0.0,
        'Capital ($/kW)' : 1280,
        'Capacity Factor' : 0.22
    },
    wind_1 : {
        'Nameplate Capacity (kW)' : 180000,
        'Fixed O&M ($/kW)': 36.0,
        'Variable O&M ($/kWh)' : 2.7/1000,
        'Capital ($/kW)' : 2005,
        'Capacity Factor' : 0.35
    },
    wind_2 : {
        'Nameplate Capacity (kW)' : 453000,
        'Fixed O&M ($/kW)': 36.0,
        'Variable O&M ($/kWh)' : 2.7/1000,
        'Capital ($/kW)' : 2005,
        'Capacity Factor' : 0.35
    }
}

for generator, info in lcoe_info_dict.items():
    capital_cost_year_one = info['Capital ($/kW)'] * (info['Nameplate Capacity (kW)'] * 1000)
    lifetime_years = 25
    discount_rate = 0.07        # AEMC uses 6-8.5% for all technologies

    numerator = 0.0
    denominator = 0.0
    capital_year_n = capital_cost_year_one
    for year in range(0, lifetime_years):
        capital_year_n = (capital_cost_year_one) * (year == 0)
        kwh_in_year_n = (info['Capacity Factor'] * info['Nameplate Capacity (kW)'] * (365*24))
        numerator += (capital_year_n + info['Fixed O&M ($/kW)']*info['Nameplate Capacity (kW)'] + info['Variable O&M ($/kWh)']*kwh_in_year_n) / ((1 + discount_rate)**year)
        denominator += (kwh_in_year_n) / ((1 + discount_rate)**year)

    final = numerator / denominator

    print(final)

NameError: name 'solar_1' is not defined

In [6]:
# At this point: loop through the loads, create a big df with results for each
# Key results at this point: percentage mix of each generator, matching percent.
filepath = LOAD_DATA_DIR / load_filename
load_data, start_date, end_date = import_load_data.get_load_data(filepath, LOAD_DATETIME_COL_NAME, LOAD_COL_NAME, DAY_FIRST)
load_data = load_data / 1000    # convert to MWh
load_data = load_data[load_data.index >= '2020-01-01 00:00:00']

one_yr_gen = gen_data.iloc[:24*365].copy().sum()
one_yr_load = load_data.iloc[:24*365].copy().sum()
gen_data_test = gen_data.copy()

for gen in gen_data_test.columns:
    gen_data_test[gen] = gen_data_test[gen] * (one_yr_load['Load'] / one_yr_gen[gen])

gen_info = {
    wind_1 : 75,
    solar_1 : 65
}

combined_data = pd.concat([load_data, gen_data_test, price_data, emissions_intensity], axis='columns')
combined_data = combined_data.dropna(how='any', axis='rows')

RETAIL_TARIFF_DETAILS = {'a': 'b'}
combined_data_firming = firming_contracts.choose_firming_type(
    FIRMING_CONTRACT_TYPE, combined_data, [LOAD_REGION], EXPOSURE_BOUND_UPPER, EXPOSURE_BOUND_LOWER, RETAIL_TARIFF_DETAILS
)
opt_hybrid, percentages = hybrid.create_hybrid_generation(
    contract_type, 
    redefine_period,  
    contract_amount, 
    combined_data_firming, 
    LOAD_REGION, 
    gen_info, 
    time_series_interval, 
    matching_percentile
)

Some missing data found. Filled with zeros.



In [7]:
bill = bill_calc.calculate_bill(
    df=combined_data_firming, 
    settlement_period=settlement_period, 
    contract_type=contract_type, 
    load_region=LOAD_REGION, 
    strike_price=strike_price, 
    lgc_buy_price=lgc_buy_price, 
    lgc_sell_price=lgc_sell_price, 
    shortfall_penalty=shortfall_penalty, 
    guaranteed_percent=guaranteed_percent, 
    excess_price=excess_price, 
    indexation=indexation, 
    index_period=index_period, 
    floor_price=floor_price
)




In [8]:
combined_data_firming.head(10)

,Load,DDSF1: PHOTOVOLTAIC FLAT PANEL,SMCSF1: PHOTOVOLTAIC FLAT PANEL,MEWF1: WIND - ONSHORE,COOPGWF1: WIND - ONSHORE,RRP: QLD1,AEI: QLD1,Firming price: QLD1,Hybrid,Contracted Energy,Strike Price (Indexed),Price,PPA Settlement,PPA Value,Excess Price,Excess Energy,Excess Energy Revenue
DateTime,,,,,,,,,,,,,,,,,
2020-01-01 01:00:00,0.12408,0.000000,0.000000,0.246866,0.442636,51.169856,0.857502,51.169856,0.043695,0.043695,124.0,72.830144,3.182329,5.418207,65,0.0,0.0
2020-01-01 02:00:00,0.12390,0.000000,0.000000,0.148081,0.457348,51.729121,0.856617,51.729121,0.026210,0.026210,124.0,72.270879,1.894239,3.250073,65,0.0,0.0
2020-01-01 03:00:00,0.12568,0.000000,0.000000,0.176148,0.392468,51.434404,0.857574,51.434404,0.031178,0.031178,124.0,72.565596,2.262458,3.866086,65,0.0,0.0
2020-01-01 04:00:00,0.12430,0.000000,0.000000,0.180930,0.334884,48.489593,0.860999,48.489593,0.032025,0.032025,124.0,75.510407,2.418193,3.971054,65,0.0,0.0
2020-01-01 05:00:00,0.12398,0.029593,0.000000,0.021875,0.327957,48.364535,0.861482,48.364535,0.028227,0.028227,124.0,75.635465,2.134940,3.500111,65,0.0,0.0
2020-01-01 06:00:00,0.12028,0.268627,0.308064,0.000000,0.339460,34.250995,0.811133,34.250995,0.221080,0.120280,124.0,89.749005,10.795010,14.914720,65,0.0,0.0
2020-01-01 07:00:00,0.11074,0.869427,1.625326,0.000000,0.352432,31.320757,0.744420,31.320757,0.715538,0.110740,124.0,92.679243,10.263299,13.731760,65,0.0,0.0
2020-01-01 08:00:00,0.11246,1.555565,3.215264,0.000000,0.259537,16.995515,0.723322,20.000000,1.280230,0.112460,124.0,107.004485,12.033724,13.945040,65,0.0,0.0
2020-01-01 09:00:00,0.11882,2.016303,4.279525,0.000000,0.191500,17.272880,0.706860,20.000000,1.659418,0.118820,124.0,106.727120,12.681316,14.733680,65,0.0,0.0


In [9]:
bill

,PPA Value,PPA Settlement,Firming Costs,Revenue from on-sold RE,Revenue from excess LGCs,Cost of shortfall LGCs,Shortfall Payments Received,Total
DateTime,,,,,,,,
2020-12-31,432418.231414,313882.440608,98768.210911,-0.0,0.0,44573.407945,-31838.148532,857804.142346
2021-12-31,0.000000,0.000000,4.007367,-0.0,0.0,3.743145,-2.673675,5.076837
